In [1]:
import os
from torch.utils.data import DataLoader
import torch
import ipdb

from tqdm import tqdm

from trajdata import AgentBatch, AgentType, UnifiedDataset
from trajdata.visualization.interactive_animation import (
    InteractiveAnimation,
    animate_agent_batch_interactive,
)

import sys

sys.path.append(os.path.join(os.getcwd(), '..'))

from  trajdata_utils.interactive_vis import plot_agent_batch_interactive
from trajdata_utils.vis import plot_agent_batch, plot_scene_batch
import copy

In [2]:
# See below for a list of already-supported datasets and splits.
torch.manual_seed(5)
dataset = UnifiedDataset(
    desired_data=["eupeds_eth", "eupeds_hotel", "eupeds_univ", "eupeds_zara1", "eupeds_zara2"],
    data_dirs={  # Remember to change this to match your filesystem!
        "eupeds_eth": "~/traj_dataset/eth-uty",
        "eupeds_hotel": "~/traj_dataset/eth-uty",
        "eupeds_univ": "~/traj_dataset/eth-uty",
        "eupeds_zara1": "~/traj_dataset/eth-uty",
        "eupeds_zara2": "~/traj_dataset/eth-uty",
    },
    history_sec =(0.0,  0.0), 
    future_sec =(15.9,  15.9),
    desired_dt=0.1,
    # centric='scene'
)
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=dataset.get_collate_fn(),
    num_workers=0, # This can be set to 0 for single-threaded loading, if desired.
)
data = iter(dataloader)
batch = next(data)

In [3]:
custom_batch = copy.deepcopy(batch)

In [4]:
custom_batch.num_neigh = torch.tensor([1])
custom_batch.neigh_hist = batch.neigh_hist[:,0:1,:,:]
custom_batch.neigh_fut = batch.neigh_fut[:,0:1,:,:]

start = torch.tensor([10.0, 0.0])
goal = torch.tensor([0.0,0.0])
custom_batch.neigh_hist.position = start
custom_batch.neigh_fut.position = torch.cat([torch.linspace(start[0], goal[0], 159).unsqueeze(1),
                                           torch.linspace(start[1], goal[1], 159).unsqueeze(1)], axis=1)

In [5]:

plot_agent_batch_interactive(custom_batch, batch_idx=0, cache_path=dataset.cache_path)

In [6]:
batch.neigh_hist[:,0:1,:,:].shape

torch.Size([1, 1, 1, 8])

In [7]:
custom_batch.neigh_hist.position = torch.tensor([1.0,1.0])

In [8]:
custom_batch.neigh_fut.position = torch.cat([torch.linspace(start[0], goal[0], 159).unsqueeze(1),
                                           torch.linspace(start[0], goal[0], 159).unsqueeze(1)], axis=1)

In [9]:
torch.save(custom_batch, 'val_data.pt')